In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 导入数据集
from Toolkit import *
import sys  
sys.path.append('/root/zqh/filter_test')
from FingerPrint_5_quick import *
from United_model import *

Pathlist = [
    '/root/zqh/NewDataSet/BCG_ZQH2.pt',
    '/root/zqh/NewDataSet/BCG_QJF1.pt',
    '/root/zqh/NewDataSet/BCG_WCM1.pt',
    '/root/zqh/NewDataSet/New_data_pa1.pt',
    '/root/zqh/NewDataSet/New_data_pa2.pt',
    '/root/zqh/NewDataSet/New_data_pa3.pt',
    '/root/zqh/NewDataSet/New_data_pa5.pt',
    '/root/zqh/NewDataSet/New_data_pa6.pt',
    '/root/zqh/NewDataSet/New_data_pa7.pt',
    '/root/zqh/NewDataSet/New_data_pa8.pt',
    '/root/zqh/NewDataSet/New_data_pa9.pt',
    '/root/zqh/NewDataSet/New_data_pa10.pt',
    '/root/zqh/NewDataSet/New_data_pa11.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa12.pt',
    '/root/zqh/NewDataSet/New_data_pa13.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa14.pt',
    '/root/zqh/NewDataSet/New_data_pa15.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa16.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa17.pt',
    '/root/zqh/NewDataSet/New_data_pa18.pt',
    '/root/zqh/NewDataSet/New_data_pa19.pt',
    '/root/zqh/NewDataSet/New_data_pa20.pt',
    '/root/zqh/NewDataSet/New_data_pa21.pt',
    '/root/zqh/NewDataSet/New_data_pa22.pt',
    '/root/zqh/NewDataSet/New_data_pa23.pt',
    '/root/zqh/NewDataSet/New_data_pa24.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa25.pt',
    '/root/zqh/NewDataSet/New_data_pa26.pt',
    '/root/zqh/NewDataSet/New_data_pa27.pt',
    '/root/zqh/NewDataSet/New_data_pa28.pt',
    '/root/zqh/NewDataSet/New_data_pa29.pt',
    '/root/zqh/NewDataSet/New_data_pa30.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa31.pt',
    '/root/zqh/NewDataSet/New_data_pa32.pt', # poor
    '/root/zqh/NewDataSet/New_data_pa33.pt',
    '/root/zqh/NewDataSet/New_data_pa34.pt',
    '/root/zqh/NewDataSet/New_data_pa35.pt',
    '/root/zqh/NewDataSet/New_data_pa36.pt',
    '/root/zqh/NewDataSet/New_data_pa37.pt',
    '/root/zqh/NewDataSet/New_data_pa38.pt',
    '/root/zqh/NewDataSet/New_data_pa39.pt',
    '/root/zqh/NewDataSet/New_data_pa40.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_caoan615.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_dj613.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_dxt613.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_ltm613.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_qjf612.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_rrx613.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_sample1.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_sjj612.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_tt612.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_wcm612.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_wg613.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_whd612.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_wxy.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_zj612.pt',
    # '/root/zqh/BCGDataSet/modify_extract_Single_resolution_zqh1.pt',
    '/root/zqh/BCGDataSet/modify_extract_Single_resolution_zzp612.pt',
]
oneperson_begin = 0
oneperson_end = 30
oneperson_nums = oneperson_end - oneperson_begin
persons = len(Pathlist)
data = get_ResUnet_data(Pathlist=Pathlist, oneperson_begin=oneperson_begin, oneperson_end=oneperson_end)[:,:,:].detach()
label = torch.zeros(oneperson_nums*persons)
for i in range(persons):
    label[i*oneperson_nums:(i+1)*oneperson_nums] = i

# 数据对齐
Unite_model = torch.load('/root/zqh/Save_Model/United_model_device.pth').cuda().eval()
feature1, ans, feature2 = Unite_model(data.cuda())
features = feature2
data = features.detach()

def splitDataSet(data, label, persons, oneperson_nums): # 要求输入 shape: N x
    X_train = data[0:20,:]
    X_test = data[20:30,:]
    y_train = label[0:20]
    y_test = label[20:30]
    for i in range(1,persons):
        X_train = torch.cat([X_train, data[oneperson_nums*i:oneperson_nums*i+20,:]],dim=0)
        X_test = torch.cat([X_test, data[oneperson_nums*i+20:oneperson_nums*i+30,:]],dim=0)
        y_train = torch.cat([y_train, label[oneperson_nums*i:oneperson_nums*i+20]],dim=0)
        y_test = torch.cat([y_test, label[oneperson_nums*i+20:oneperson_nums*i+30]],dim=0)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = splitDataSet(data.cpu().squeeze(1), label, persons, oneperson_nums)
# 导入数据集

# 训练随机森林
model = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, random_state=42)
model.fit(X_train, y_train)

def MyRandomForest(persons, X_train, X_test, y_train, y_test, k):
    right = 0
    oneperson_nums = 10
    for i in range(X_test.shape[0]):
        # 切分数据
        databegin = i//oneperson_nums*oneperson_nums
        if k==1:
            Kset = X_test[i:i+1,:]
        else:
            _,remainer,_,_ = train_test_split(X_test[databegin:databegin+oneperson_nums,:],
                                              X_test[databegin:databegin+oneperson_nums,:],
                                              test_size=(k-1)/oneperson_nums,
                                              random_state=i)
            Kset = torch.cat([X_test[i:i+1,:],remainer],dim=0)
        # 测试
        record = torch.zeros(persons)
        predicted_list = model.predict(Kset)
        for j in range(k):
            record[int(predicted_list[j])] += 1
        maxvalue,maxindex = torch.max(record,dim=0)
        if maxindex == y_test[i]:
            right+=1
    return right/y_test.shape[0]

# 结果
list = []
k_range = 9
for i in range(1, k_range):
    ans = MyRandomForest(persons, X_train, X_test, y_train, y_test, k=i)
    list.append(ans)
arraylist = np.array(list)
print(arraylist)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
